## Part 1: Preprocessing

In [117]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [118]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [119]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]


In [120]:
# Create a list of at least 10 column names to use as X data
columns = ['Age', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'TotalWorkingYears', 'YearsSinceLastPromotion']

# Create X_df using your selected columns
X_df = attrition_df[columns]

# Show the data types for X_df
X_df.dtypes


Age                        int64
DistanceFromHome           int64
Education                  int64
EnvironmentSatisfaction    int64
HourlyRate                 int64
JobInvolvement             int64
JobLevel                   int64
JobSatisfaction            int64
TotalWorkingYears          int64
YearsSinceLastPromotion    int64
dtype: object

In [121]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=42)


In [122]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

# They are all already numeric.

In [123]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [124]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
training_encoded = encoder.transform(y_train[['Department']])
test_encoded = encoder.transform(y_test[['Department']])

training_encoded

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [125]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
at_training_encoded = attrition_encoder.transform(y_train[['Attrition']])
at_test_encoded = attrition_encoder.transform(y_test[['Attrition']])

at_training_encoded

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]])

## Create, Compile, and Train the Model

In [126]:
# Find the number of columns in the X training data
num_features = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(num_features,))

# Create at least two shared layers
shared_layer1 = layers.Dense(units=64, activation='relu')(input_layer) 
shared_layer2 = layers.Dense(units=128, activation='relu')(shared_layer1)

In [127]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden_layer = layers.Dense(units=32, activation='relu', name='Department_Hidden')(shared_layer2)

# Create the output layer
department_output_layer = layers.Dense(units=3, activation='sigmoid', name='Department_Output')(department_hidden_layer)


In [128]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden_layer = layers.Dense(units=32, activation='relu', name='Attrition_Hidden')(shared_layer2)

# Create the output layer
attrition_output_layer = layers.Dense(units=2, activation='sigmoid', name='Attrition_Output')(attrition_hidden_layer)


In [129]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output_layer, attrition_output_layer])

# Compile the model
model.compile(optimizer='adam',
              loss={'Department_Output': 'binary_crossentropy', 'Attrition_Output': 'binary_crossentropy'},
              metrics={'Department_Output': 'accuracy', 'Attrition_Output': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_10      │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_30 (Dense)    │ (None, 64)        │        704 │ input_layer_10[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_31 (Dense)    │ (None, 128)       │      8,320 │ dense_30[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Department_Hidden   │ (None, 32)        │      4,128 │ dense_31[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Attrition_Hidden    │ (None, 32)        │      4,128 │ dense_31[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Department_Output   │ (None, 3)         │         99 │ Department_Hidde… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Attrition_Output    │ (None, 2)         │         66 │ Attrition_Hidden… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [131]:
# Train the model
history=model.fit(
    X_train_scaled, 
    {
        'Department_Output': training_encoded,
        'Attrition_Output': at_training_encoded
    },
    epochs=50,
    batch_size=32,
    validation_data=(
        X_test_scaled, 
        {
            'Department_Output': test_encoded, 
            'Attrition_Output': at_test_encoded
        }
    )
)


Epoch 1/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - Attrition_Output_accuracy: 0.7167 - Attrition_Output_loss: 0.5949 - Department_Output_accuracy: 0.6167 - Department_Output_loss: 0.6096 - loss: 1.2048 - val_Attrition_Output_accuracy: 0.8696 - val_Attrition_Output_loss: 0.3922 - val_Department_Output_accuracy: 0.6522 - val_Department_Output_loss: 0.4926 - val_loss: 0.8741
Epoch 2/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - Attrition_Output_accuracy: 0.8279 - Attrition_Output_loss: 0.4607 - Department_Output_accuracy: 0.6668 - Department_Output_loss: 0.4679 - loss: 0.9288 - val_Attrition_Output_accuracy: 0.8696 - val_Attrition_Output_loss: 0.3858 - val_Department_Output_accuracy: 0.6522 - val_Department_Output_loss: 0.4820 - val_loss: 0.8575
Epoch 3/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - Attrition_Output_accuracy: 0.8254 - Attrition_Output_loss: 0.4374 - Department_Output_accuracy: 0.6303 - Department_Output_loss: 0.4704 - loss: 0.9079 - val_Attrition_Output_accuracy: 0.8696 - val

In [137]:
# Evaluate the model with the testing data
evaluation = model.evaluate(X_test_scaled, 
                            {'Department_Output': test_encoded, 
                             'Attrition_Output': at_test_encoded})

# Print the evaluation results
print("Model evaluation complete.")
print(f"Loss: {evaluation[0]}")
print(f"Department Output - Loss: {evaluation[1]}, Accuracy: {evaluation[3]}")
print(f"Attrition Output - Loss: {evaluation[2]}, Accuracy: {evaluation[4]}")


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - Attrition_Output_accuracy: 0.8434 - Attrition_Output_loss: 0.6297 - Department_Output_accuracy: 0.5973 - Department_Output_loss: 0.7305 - loss: 1.3558 
Model evaluation complete.
Loss: 1.2888809442520142
Department Output - Loss: 0.7326938509941101, Accuracy: 0.8505434989929199
Attrition Output - Loss: 0.5849449038505554, Accuracy: 0.614130437374115


In [136]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {evaluation[3]}")
print(f"Attrition Accuracy: {evaluation[4]}")

Department Accuracy: 0.8505434989929199
Attrition Accuracy: 0.614130437374115


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Chat says that "Accuracy is a common metric, but it may not always be the best choice, especially for imbalanced datasets". I do believe our set was a bit imbalanced so perhaps another metric would be good perhaps in addition to accuracy.
2. I chose to use Sigmoid as this is what was recommended. I an unsure of why this is the better choice.
3. I think I would have to spend more time in looking at the model and inputs. Doing more processing and discovery on the parameters. 